In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import random
from PIL import Image 
import os
import sklearn.metrics as skm
import csv

### 獲取要隱藏的訊息

1. 逐字讀取原始訊息
2. 轉換為ascii碼
3. ascii碼轉換為二進制(0bxxxxxxx)的字串
4. replace()替換0b為空
5. 七位數二進制填充為八位數

In [2]:
def getMessage(txtPath):
    byteNumber=0
    with open(txtPath,"rb")  as file:
        txt = file.read()
        byteNumber=len(txt)
        charMessage=""
        for index in range(len(txt)):
            charMessage=charMessage+""+str(bin(txt[index]).replace('0b','')).zfill(7)
            if(index<10):
                print('第'+str(index)+'位為:  '+chr(txt[index])+', 二進制為: '+str(bin(txt[index]).replace('0b','')).zfill(7))
            else:
                print('第'+str(index)+'位為: '+chr(txt[index])+', 二進制為: '+str(bin(txt[index]).replace('0b','')).zfill(7))
    print("輸出為："+charMessage) 
    print("字元數："+str(byteNumber)) 
    print("位元數："+str(len(charMessage))) 
    return charMessage

In [3]:
message=getMessage("../message/1024message.txt")

第0位為:  N, 二進制為: 1001110
第1位為:  I, 二進制為: 1001001
第2位為:  U, 二進制為: 1010101
第3位為:  1, 二進制為: 0110001
第4位為:  1, 二進制為: 0110001
第5位為:  1, 二進制為: 0110001
第6位為:  2, 二進制為: 0110010
第7位為:  C, 二進制為: 1000011
第8位為:  r, 二進制為: 1110010
第9位為:  y, 二進制為: 1111001
第10位為: p, 二進制為: 1110000
第11位為: t, 二進制為: 1110100
第12位為: o, 二進制為: 1101111
第13位為: g, 二進制為: 1100111
第14位為: r, 二進制為: 1110010
第15位為: a, 二進制為: 1100001
第16位為: p, 二進制為: 1110000
第17位為: h, 二進制為: 1101000
第18位為: y, 二進制為: 1111001
第19位為: N, 二進制為: 1001110
第20位為: I, 二進制為: 1001001
第21位為: U, 二進制為: 1010101
第22位為: 1, 二進制為: 0110001
第23位為: 1, 二進制為: 0110001
第24位為: 1, 二進制為: 0110001
第25位為: 2, 二進制為: 0110010
第26位為: C, 二進制為: 1000011
第27位為: r, 二進制為: 1110010
第28位為: y, 二進制為: 1111001
第29位為: p, 二進制為: 1110000
第30位為: t, 二進制為: 1110100
第31位為: o, 二進制為: 1101111
第32位為: g, 二進制為: 1100111
第33位為: r, 二進制為: 1110010
第34位為: a, 二進制為: 1100001
第35位為: p, 二進制為: 1110000
第36位為: h, 二進制為: 1101000
第37位為: y, 二進制為: 1111001
第38位為: N, 二進制為: 1001110
第39位為: I, 二進制為: 1001001
第40位為: U, 二進制為: 1010101
第41位為: 1, 二進制為: 

In [4]:
#取得單一字元二元序列
def getACharMessage(message,index):
    return message[(index*7):(index*7)+7]

### 取得原始影像資料

1. 將影像轉成灰階
2. 轉換成二維像素陣列

In [16]:
imageNamePath = '../database/resizeImageData.txt'
imageNameArray=[]
imageLength=0
file = open(imageNamePath, 'r')
for name in file.readlines():
    imageNameArray.append('../database'+name[1:-1])
file.close()
imageLength=len(imageNameArray)
print("照片數量: "+str(imageLength))

照片數量: 100


### 公用函數

1. mod(x,y) 取餘數
2. hideMessagePixel(testImage,message,afterSrc)
3. psnr(originImage,hidedImage) 計算 psnr
4. hxx_forward(x, y) 計算相對熵

In [6]:
# 取餘數
def mod(x,y):
    return x%y

In [17]:
def hideMessagePixel(testImage,message,afterSrc,ptr):
    #獲取圖片的寬和高
    width,height= testImage.size[0],testImage.size[1]
    if(ptr):
        print("width:"+str(width))
        print("height:"+str(height))

    count = 0

    #獲取需要隱藏的信息 
    messageLength = len(message)
    for h in range(height):
        for w in range(width):
            pixel = testImage.getpixel((w,h))
            # a=pixel[0]
            # b=pixel[1]
            # c=pixel[2]
            
            if count == messageLength:
                break
            
            pixel= pixel-mod(pixel,2)+int(message[count])
            # print('Pixel('+str(w)+','+str(h)+')= '+str(pixel),'AfterHiddedPixel= '+str((a,b-mod(b,2)+int(message[count]),c-mod(c,2)+int(message[count]))))

            count+=1
            if count == messageLength:
                testImage.putpixel((w,h),pixel) 
                break
            # b =b-mod(b,2)+int(message[count])
            # count+=1 
            # if count == messageLength:
            #     testImage.putpixel((w,h),(a,b,c)) 
            #     break
            # c= c-mod(c,2)+int(message[count])
            # count+=1
            # if count == messageLength:
            #     testImage.putpixel((w,h),(a,b,c))
            #     break
            if count % 3 == 0:
                testImage.putpixel((w,h),pixel)
    testImage.save(afterSrc)

In [18]:
# 計算 psnr
def psnr(originImage,hidedImage):
   diffrence = originImage-hidedImage
   mse = np.mean(np.square(diffrence))
   psnr = 10 * np.log10(255 * 255 / mse)
   return(psnr)

In [19]:
#計算相對熵
def hxx_forward(x, y):
    return skm.mutual_info_score(x, y)

### 單一影像生成

#### 輸入

 testImage: 單一圖像

 message: 隱藏訊息

 prt: 是否顯示提示
 
 path: 隱藏圖像儲存路徑

In [20]:
def LSBHidingSingleImage(imagePath,message,ptr,afterPath):
    image=Image.open(imagePath) 
    hideMessagePixel(image,message,afterPath,ptr)

    LSBImage = cv2.imread(afterPath)
    LSBImage = cv2.cvtColor(LSBImage,cv2.COLOR_BGR2GRAY) 
    grayscaleImage = cv2.imread(imagePath)
    grayscaleImage = cv2.cvtColor(grayscaleImage,cv2.COLOR_BGR2GRAY) 

    if(ptr):
        print('PSNR: '+str(psnr(grayscaleImage,LSBImage)))

    x = np.reshape(grayscaleImage, -1)
    y = np.reshape(LSBImage, -1)
    
    if(ptr):
        print("相對熵= "+str(hxx_forward(x, y)))

    return psnr(grayscaleImage,LSBImage),hxx_forward(x, y)
    

In [21]:
LSBHidingSingleImage(imageNameArray[0],message,True,'../images/testAfterLSB.jpg')

width:224
height:224
PSNR: 34.71156833867159
相對熵= 1.765895746060459


(34.71156833867159, 1.765895746060459)

In [22]:
LSBHidingSingleImage(imageNameArray[0],message,False,'../images/testAfterLSB.jpg')

(34.71156833867159, 1.765895746060459)

### 多個影像生成

#### 輸入

 message: 隱藏訊息

 prt: 是否顯示提示
 
 dirPath: 隱藏圖像儲存路徑

In [23]:
def writeCSV(array,fileName):
    with open(fileName, 'w', newline='') as csvfile:
        # 以空白分隔欄位，建立 CSV 檔寫入器
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(['LSB'])
        for number in array:
            writer.writerow([number])
    csvfile.close()

In [24]:
def LSBHidingMultipleImage(imageNameArray,message,ptr):
    psnrArray=[]
    hxxArray=[]

    for index,imageName in enumerate(imageNameArray):
        print(str(index+1))
        psnr,hxx=LSBHidingSingleImage(imageName,message,ptr,'./database/'+str(index+1)+'LSBHide.jpg')
        psnrArray.append(psnr)
        hxxArray.append(hxx)

    if(ptr):
        print(psnrArray)
        print(hxxArray)
    
    writeCSV(psnrArray,'./PSNR.csv')
    writeCSV(hxxArray,'./hxx.csv')

In [25]:
LSBHidingMultipleImage(imageNameArray,message,True)

1
width:224
height:224
PSNR: 34.71156833867159
相對熵= 1.765895746060459
2
width:224
height:224
PSNR: 39.56530201405629
相對熵= 2.9003859942416716
3
width:224
height:224
PSNR: 39.44948804241407
相對熵= 2.621570620908182
4
width:224
height:224
PSNR: 34.55592485080784
相對熵= 2.3087575803579106
5
width:224
height:224
PSNR: 38.014665327626204
相對熵= 2.995895288391404
6
width:224
height:224
PSNR: 35.40038929500364
相對熵= 2.0260986715487337
7
width:224
height:224
PSNR: 38.78467793813938
相對熵= 2.8376592261541838
8
width:224
height:224
PSNR: 40.27116163798613
相對熵= 3.1164350414835065
9
width:224
height:224
PSNR: 34.71757523345266
相對熵= 2.163069133970346
10
width:224
height:224
PSNR: 43.504836321891304
相對熵= 2.3779239292150423
11
width:224
height:224
PSNR: 41.8358291607948
相對熵= 3.3202740228562533
12
width:224
height:224
PSNR: 40.14767036332066
相對熵= 2.589964775946784
13
width:224
height:224
PSNR: 34.34559852706246
相對熵= 2.5072034005243937
14
width:224
height:224
PSNR: 33.48308323899879
相對熵= 1.7620163432283695
15
wi